<a href="https://colab.research.google.com/github/aparnasree2020/sampledata_casws/blob/main/Run_XGboost_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install packages

In [1]:
# Comment after installation
#install.packages('gbm')
#install.packages('Hmisc')


# Load Libraries

In [2]:
#library(dplyr)
library(tidyverse)
#require(xgboost)
library(gbm)
#library(measures)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.4.1     ✔ purrr   1.0.1
✔ tibble  3.1.8     ✔ dplyr   1.1.0
✔ tidyr   1.3.0     ✔ stringr 1.4.1
✔ readr   2.1.4     ✔ forcats 1.0.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



ERROR: ignored

# Load CSV datafile

In [ ]:
myfile <- "https://raw.github.com/aparnasree2020/sampledata_casws/master/Cleaned_data.csv"
#csv_filename = 'gbm_data_rpm_2023_100K_v2.csv'
gbm_data <- read.csv(file =myfile, header = TRUE )



## Split data into train and test at 80/20 split

In [ ]:
size <- floor(0.80 * nrow(gbm_data))
set.seed(42)
train_idx <- sample(seq_len(nrow(gbm_data)), size = size)
train <- gbm_data[train_idx,]
test <- gbm_data[-train_idx,]

x_train <- train[, !names(train) %in% "claim_count_pd"]
y_train <- train$claim_count_pd
x_test <- test[, !names(test) %in% "claim_count_pd"]
y_test <- test$claim_count_pd

# Fit XGBoost model 

The hyperparameters used are:

*   eta (learning rate): Controls the step-size at which the optimizer makes updates to the weights.
*   max_depth: The maximum depth of a tree.
* min_child_weight: The minimum weight required for a child to be split.
* subsample: The fraction of the training data used in each iteration.
* colsample_bytree: The fraction of the columns used in each iteration.
* gamma: The minimum loss reduction required to make a split.
* alpha: L1 regularization parameter.
* lambda: L2 regularization parameter.
* nthread: The number of parallel threads used to run XGBoost.
* seed: The random seed used to initialize the weights.

These hyperparameters can be fine-tuned to achieve the best performance for a particular problem. The values given here are just an example and may need to be adjusted for different datasets.

In [ ]:
# Train the model
bst <- xgboost(data = as.matrix(train_data), label = train_labels, nrounds = 100, objective = "binary:logistic",
               eta = 0.3,
               max_depth = 6,
               min_child_weight = 1,
               subsample = 0.8,
               colsample_bytree = 0.8,
               gamma = 0,
               alpha = 0,
               lambda = 1,
               nthread = 4,
               seed = 123)




#predict output for test data

In [ ]:
# Predict the classes for the test data
pred_y_test <- predict(bst, as.matrix(test_data))

pred_y_test_bin = ifelse(pred_y_test > 0.5, 1, 0)

In [ ]:
dataout <- data.frame(pred_y_test_bin, y_test)
write.csv(dataout, "XGboost_hyperparam.csv", row.names=FALSE)

# End

Unused code below

In [ ]:
#library(Hmisc)
#hist.data.frame(gbm_data$claim_count_pd)